In [1]:
import geopandas as gpd
import pickle
from sklearn.neighbors import BallTree
import numpy as np
from shapely.geometry import LineString
%matplotlib inline
import matplotlib.pyplot as plt
from itertools import compress
import pandas as pd

root_path = "D:/GeoData/"
Main_CRS = "EPSG:27700"

In [2]:
def load_obj(name ):
    with open(root_path + 'WorkingData/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [3]:
def save_obj(name , obj, extra=""):
    with open(root_path + 'WorkingData/' + extra + name + '.pkl', 'wb+') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [4]:
def distance(src_points, candidates, source_name, out_name, keep_d_n_g=[True,True]):
    
    dist_name = "Distance_To_Nearest_" + out_name + "(m)"
    name_name = "Name_of_Nearest_" + out_name
    geom_name = "Geometry_of_Nearest_" + out_name
    
    candidates = candidates.reset_index(drop=True)
    
    dist_df = src_points.geometry.apply(lambda g: candidates.distance(g))
    min_distance = list(dist_df.min(axis=1).round(0))
    min_name = list(candidates[source_name].iloc[dist_df.idxmin(axis=1)])
    min_geom = list(candidates["geometry"].iloc[dist_df.idxmin(axis=1)])
    
    df = pd.DataFrame(list(zip(min_distance, min_name, min_geom)),
                      columns =[dist_name, name_name, geom_name],
                      index =src_points.index)
    
    return df

In [5]:
def get_nearest_point(src_points, #Source gdf, the gdf with the points you want to attach distances to
                      candidates, #gdf containing the list of points you want to get the distances to
                      k_neighbors=1 #Return just the closest point
                     ):
    """Find nearest neighbors for all source points from a set of candidate points"""

    # Create tree from the candidate points
     # removed metric since the default is euclidian (what my coordinates use)
    tree = BallTree(candidates, leaf_size=15)

    # Find closest points and distances
    distances, indices = tree.query(src_points, k=k_neighbors)

    # Transpose to get distances and indices into arrays
    distances = distances.transpose()
    indices = indices.transpose()

    # Get closest indices and distances (i.e. array at index 0)
    # note: for the second closest points, you would take index 1, etc.
    closest = indices[0]
    closest_dist = distances[0]

    # Return indices and distances
    return (closest, closest_dist)

In [6]:
def nearest_neighbor_point(left_gdf, #source data
                         right_gdf, #points data
                         right_col_name, #in the points data, what column contains the name of the points
                         outname, #in the output data what do you want to refer to the points as
                         keep_n_g_d = [True, True, True], #do you want to keep the geometry, point name and distance
                         merge=False #Make a new output or merge onto left_gdf
                        ):
    """
    For each point in left_gdf, find closest point in right GeoDataFrame and return them.
    """
    
    suffix = "Nearest_" + outname
        
    if len(right_gdf)>0:

        # Ensure that index in right gdf is formed of sequential numbers
        left = left_gdf.copy().reset_index(drop=False)
        right = right_gdf.copy().reset_index(drop=True)
        
        left_geom_col = left.geometry.name
        right_geom_col = right.geometry.name
        
        #Some of the data frames will have empty geometries so this will drop them
        left = left[~(left.is_empty)]
        right = right[~(right.is_empty)]
        
        left = left.copy().reset_index(drop=False)
        right = right.copy().reset_index(drop=False)

        left_points = np.array(left[left_geom_col].apply(lambda geom: (geom.x, geom.y)).to_list())
        right_points = np.array(right[right_geom_col].apply(lambda geom: (geom.x, geom.y)).to_list())

        # Find the nearest points
        # -----------------------
        # closest ==> index in right_gdf that corresponds to the closest point
        # dist ==> distance between the nearest neighbors (in meters)

        closest, dist = get_nearest_point(src_points=left_points, candidates=right_points)

        # Return points from right GeoDataFrame that are closest to points in left GeoDataFrame
        closest_points = right.loc[closest]

        # Ensure that the index corresponds the one in left_gdf
        left_index = left['index']
        closest_points = closest_points.reset_index(drop=True)
        closest_points = closest_points.set_index(left_index)

        #Select the columns to keep in the output dataframe
        keep = [right_col_name, 'geometry']  
        closest_points = closest_points[keep]

        #Rename the columns to keep
        closest_points = closest_points.rename(columns={right_col_name: suffix + '_name'})
        closest_points = closest_points.rename(columns={'geometry': suffix + '_geometry'})
        closest_points[suffix + '_Distance'] = dist

        closest_points = closest_points.loc[:,keep_n_g_d]

    else:
        closest_points = left_gdf.iloc[:,0:1].copy()
        closest_points[suffix + '_name'] = ""
        closest_points[suffix + '_geometry'] = ""
        closest_points = closest_points.loc[:,[suffix + '_name',suffix + '_geometry']]
        closest_points[suffix + '_Distance'] = np.nan
        closest_points = closest_points.loc[:,keep_n_g_d]
        
        
    # either add the columns to left_gdf or make a new dataframe
    if merge:
        out = left_gdf.join(closest_points)
    else:
        out = closest_points
    return out

Set the data up on a small set of maps and see which method is quicker

In [7]:
whichmap = "Milford Haven"
Mapset = load_obj("Clipped5k_"+whichmap)

left = Mapset['NSPL_gdf']
right = Mapset['RailwayStations']
oldName = "StationName"
newName = "TrainStation"

In [8]:
%%timeit -n 100
test = nearest_neighbor_point(left, right, oldName, newName)

19.6 ms ± 225 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [9]:
%%timeit -n 100
test = left.join(distance(left, right, oldName, newName))

39.1 ms ± 230 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Now try on a big dataset (all UK postcodes)

In [10]:
All_Maps = load_obj("All_Maps")

left = All_Maps['NSPL_gdf']
right = All_Maps['RailwayStations']
oldName = "StationName"
newName = "TrainStation"

In [11]:
%%timeit
test = nearest_neighbor_point(left, right, oldName, newName)

59.5 s ± 1.44 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
%%timeit
test = left.join(distance(left, right, oldName, newName))

KeyboardInterrupt: 

In [27]:
whichmap = "Euston"
Mapset = load_obj("Clipped5k_"+whichmap)

left = Mapset['NSPL_gdf']
right = Mapset['RailwayStations']
oldName = "StationName"
newName = "TrainStation"

In [28]:
sindex = right.sindex

In [29]:
for geom in left.geometry:
    potential = list(sindex.nearest(geom.bounds, num_results=5)) # get five closest geoms based on rtree
    subset = right.iloc[potential]
    closest = right.loc[subset.distance(geom).idxmin()] # get the closest feature

In [31]:
potential

[8, 15, 14, 11, 4]

In [32]:
subset

,TiplocCode,CrsCode,StationName,Easting,Northing,geometry
2091,CHRX,CHX,London Charing Cross Rail Station,530235,180455,POINT (530235.000 180455.000)
2128,BLFR,BFR,London Blackfriars Rail Station,531714,180914,POINT (531714.000 180914.000)
2127,CTMSLNK,CTK,City Thameslink Rail Station,531690,181150,POINT (531690.000 181150.000)
2122,FRNDNLT,ZFD,Farringdon (London) Rail Station,531560,181840,POINT (531560.000 181840.000)
2078,EUSTON,EUS,London Euston Rail Station,529545,182675,POINT (529545.000 182675.000)


In [30]:
closest

TiplocCode                                  CHRX
CrsCode                                      CHX
StationName    London Charing Cross Rail Station
Easting                                   530235
Northing                                  180455
geometry                   POINT (530235 180455)
Name: 2091, dtype: object

In [ ]:
#add geom to the above output so i can make nice pics

#second post on this
#https://jakevdp.github.io/PythonDataScienceHandbook/01.07-timing-and-profiling.html

#https://github.com/geopandas/geopandas/issues/1455  #this one may prove the most useful
#https://geoffboeing.com/2016/10/r-tree-spatial-index-python/